### Cleaning and EDA Comments

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("comments") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.cores", "3") \
    .config("spark.executor.instances", "2") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1716512352629_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
import matplotlib.pyplot as plt
import pandas as pd

from pyspark.sql.functions import (
    col, udf, to_timestamp, desc, year, count, round, 
    monotonically_increasing_id, concat, lit, explode)
from pyspark.sql.functions import (
    coalesce, col, explode, from_json, length, from_unixtime,
    regexp_replace, split, size)

import pyspark.sql.functions as F

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# reading in first, partitioning for parallelization
data = spark.read.parquet("s3://131313113finalproject/ffcf_ft").repartition(4).persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data.printSchema()
print('Total Rows: %d' % data.count())
print('Total Columns: %d' % len(data.dtypes))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- body: string (nullable = true)
 |-- author: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- score: string (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- time: string (nullable = true)

Total Rows: 206756
Total Columns: 9

## Basic Cleaning

In [6]:
data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-------------+-----+-------+----------+------------+--------------------+-------------------+
|                body|            author|  created_utc|score|     id|   link_id|   subreddit|   author_flair_text|               time|
+--------------------+------------------+-------------+-----+-------+----------+------------+--------------------+-------------------+
|Been there at 32 ...|        yankee77wi|   1640635686|    1|hq6pwcl| t3_rpsgta|Conservative|        Conservative|2021-12-27 20:08:06|
|No.  You're insan...|   CarsomyrPlusSix|   1657393835|   -7|ifi9d18| t3_vv0nx9|Conservative|PaleoConservative...|2022-07-09 19:10:35|
|           [removed]|         [deleted]|   1661355939|    1|illvh2o| t3_wwie2k|Conservative|                null|2022-08-24 15:45:39|
|           [deleted]|         [deleted]|   1372882179|   -6|cave79h| t3_1hki8n|Conservative|                null|2013-07-03 20:09:39|
|I see your argume...|          ventorun|   1661449223|

I obviously have a lot more cleaning to do on this one. 
I am not worried about any of the other 'null' values, except for the actual comment body. Everything else I will still be able to glean information from. I am going to start with seeing how many removed comments, because I don't want to preform unnecessary operations on cols to be dropped, there are and proceeding from there.

In [6]:
# removed if different than null
print('Null Body Values:', data.filter(col('body').isNull()).count())

print('[removed] Body Values:', data.where(data['body'] == '[removed]').count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Null Body Values: 0
[removed] Body Values: 51888

In [7]:
# A ton of removed values. This could either be users removing their comments themseleves, or moderators doing so. Because the number is so large, I do want to look closer
removed_comments = data.where(data['body'] == '[removed]')
removed_comments.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+-----------+-----+-------+----------+------------+-----------------+-------------------+
|     body|   author|created_utc|score|     id|   link_id|   subreddit|author_flair_text|               time|
+---------+---------+-----------+-----+-------+----------+------------+-----------------+-------------------+
|[removed]|[deleted]| 1661355939|    1|illvh2o| t3_wwie2k|Conservative|             null|2022-08-24 15:45:39|
|[removed]|[deleted]| 1656791202|    1|iem3nnu| t3_vpquiv|Conservative|             null|2022-07-02 19:46:42|
|[removed]|[deleted]| 1674707673|    1|j5x8l4m|t3_10l835i|Conservative|             null|2023-01-26 04:34:33|
|[removed]|[deleted]| 1659528406|    1|iirsnb4| t3_weuums|Conservative|             null|2022-08-03 12:06:46|
|[removed]|[deleted]| 1656344744|    1|idxwk70| t3_vl4ztm|Conservative|             null|2022-06-27 15:45:44|
+---------+---------+-----------+-----+-------+----------+------------+-----------------+-------------------+
only showi

This might be something interesting to look back on later, to see if there are any commonalities within the deleted comments. For now, it seems like there isn't much happening here. Most of them have a score of 1 (the default assigned score) showing that the comment really had no impact on the discourse of the sub. It appears as though the majority of these were self deleted.

In [8]:
print('Before removing '"[removed]/[deleted]"': %d' % data.count())
data = data.filter((data['body'] != '[removed]') & (data['body'] != '[deleted]'))
print('After removing '"[removed]/deleted"': %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Before removing [removed]/[deleted]: 206756
After removing [removed]/deleted: 146173

In [9]:
data = data.withColumn('time', to_timestamp(data['time']))
# dropping uneccessary
data = data.drop('created_utc')

# changing score to int
data = data.withColumn('score', data['score'].cast('int'))

# Show the data
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- body: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- time: timestamp (nullable = true)

In [10]:
# most of cleaning will take place during the pipeline stage for NLP, but I still want to remove the \n
data = data.withColumn('body', regexp_replace('body', '\n', ''))

# and replace any links
data = data.withColumn('body', regexp_replace('body', 'http\S+', 'LINK'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# more recent comments
data.orderBy('time', ascending=False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-------+----------+------------+--------------------+-------------------+
|                body|              author|score|     id|   link_id|   subreddit|   author_flair_text|               time|
+--------------------+--------------------+-----+-------+----------+------------+--------------------+-------------------+
|There is, it's 24...|        Spring_Choco|    4|ke2fgl2|t3_18m7x6p|Conservative|                null|2023-12-19 18:15:59|
|I literally can’t...|   Commander-Grammar|   -1|kcnjgs7|t3_18e3isp|Conservative|        Conservative|2023-12-09 17:10:59|
|We should stick t...|           wabbott82|  151|kcl5g2n|t3_18e3isp|Conservative|      British Accent|2023-12-09 02:55:59|
|The title saying ...|    Angry-Wombat1871|    5|k9kjmrm|t3_17wxuid|Conservative|                null|2023-11-16 23:38:59|
|Being pro-life wi...|       gh0stwriter88|    8|k8lk6sx|t3_17rg856|Conservative|        Conservative|2023-11-10 03:21:59|
|A clump of cell

In [12]:
# lots of null times.. this seems to be an issue within the zst data
data.orderBy('time', ascending=True).show()

null_time = data.filter(col('time').isNull())
null_time_count = null_time.count()

print('Null time count:', null_time_count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-------+----------+------------+--------------------+----+
|                body|              author|score|     id|   link_id|   subreddit|   author_flair_text|time|
+--------------------+--------------------+-----+-------+----------+------------+--------------------+----+
|        Primary him.|   PerfectlyCalmDude|    1|jjgtdcw|t3_13c8tad|Conservative|Pragmatic Constit...|null|
|>The pill prevent...|           zrennetta|    6|k8g1ds0|t3_17qp9w5|Conservative|Constitutional De...|null|
|When you can't ac...|              Roku-6|    1|k5rbcgb|t3_17cmyhb|Conservative|                null|null|
|You're conflating...|            Briguy28|   33|jje4cei|t3_13c08xf|Conservative|Cascadian Conserv...|null|
|Read the article....|            roseffin|    1|jw5h4qr|t3_15qvrmj|Conservative| Fiscal Conservative|null|
|How is murdering ...|          symbiote24|   -2|jfw63aa|t3_12iwa3k|Conservative|Bill of Rights En...|null|
|Yeah I’m pro life...|    Th

I wanted to show the older comments, but there are a lot of null comments. Looking back at the zst file, it seems to be a problem stemming from there. Luckily, I have the post times, so I can estimate the time of the comment from there. It won't be accurate down to the day, but it will give a estimate.

# Filling in null times
I am going to filter out null times, join the link ids with the ids from the posts, and then join this dataframe with the full dataframe

In [13]:
# reading in post df
abortion_posts = spark.read.parquet('s3://131313113finalproject/posts_w_comments/')

# and adding on the t3_ col again, but creating a df with time
post_time = abortion_posts.withColumn("parent_id", concat(lit("t3_"), col("id"))).select("parent_id", col("date_posted").alias("estimated_time"))

post_time.printSchema()

post_time.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- parent_id: string (nullable = true)
 |-- estimated_time: timestamp (nullable = true)

+---------+-------------------+
|parent_id|     estimated_time|
+---------+-------------------+
| t3_72v18|2008-09-22 19:01:29|
| t3_ahsop|2009-12-23 10:33:52|
| t3_bcj1g|2010-03-12 13:45:03|
| t3_bjsrb|2010-03-29 17:46:07|
| t3_c45dz|2010-05-14 17:07:09|
| t3_chxh4|2010-06-22 23:27:18|
| t3_cnyoj|2010-07-10 04:58:16|
| t3_d799v|2010-08-30 15:24:32|
| t3_er128|2010-12-24 19:50:59|
| t3_f0gf5|2011-01-11 21:13:24|
| t3_f5hcf|2011-01-20 01:00:28|
| t3_g1xdt|2011-03-11 13:59:34|
| t3_g2neq|2011-03-12 17:05:08|
| t3_gf0kc|2011-03-30 21:30:32|
| t3_gufsw|2011-04-20 13:30:43|
| t3_gv102|2011-04-21 03:25:23|
| t3_gvs0r|2011-04-23 16:07:57|
| t3_j6r1v|2011-08-02 15:18:39|
| t3_kqj7z|2011-09-25 02:24:01|
| t3_l7gor|2011-10-10 20:37:47|
+---------+-------------------+
only showing top 20 rows

In [14]:
# joining
joined_null = null_time.join(post_time, 
                            null_time["link_id"] == post_time["parent_id"], 
                            how="left")

# filtering what I can see to make validation of accuracy easier
joined_null.select('estimated_time', 'link_id', 'parent_id').show()

# validated

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+----------+
|     estimated_time|   link_id| parent_id|
+-------------------+----------+----------+
|2023-05-24 01:58:44|t3_13q7u1m|t3_13q7u1m|
|2023-11-08 02:10:02|t3_17qb6ii|t3_17qb6ii|
|2023-06-26 00:49:03|t3_14j2lcn|t3_14j2lcn|
|2023-04-11 17:52:13|t3_12irndl|t3_12irndl|
|2023-04-18 15:47:17|t3_12qsp4u|t3_12qsp4u|
|2023-04-05 20:51:17|t3_12cxg11|t3_12cxg11|
|2023-04-23 23:04:34|t3_12wtrep|t3_12wtrep|
|2023-04-04 18:44:57|t3_12bsdkm|t3_12bsdkm|
|2023-04-14 10:54:07|t3_12lt8za|t3_12lt8za|
|2023-05-19 21:27:36|t3_13m8ank|t3_13m8ank|
|2023-08-12 15:12:22|t3_15p73ye|t3_15p73ye|
|2023-08-14 14:09:00|t3_15qvrmj|t3_15qvrmj|
|2023-10-10 21:46:25|t3_174x4w9|t3_174x4w9|
|2023-05-28 23:00:14|t3_13uds4n|t3_13uds4n|
|2023-04-11 17:52:13|t3_12irndl|t3_12irndl|
|2023-08-25 16:48:49|t3_1614iob|t3_1614iob|
|2023-12-18 13:49:28|t3_18l94wl|t3_18l94wl|
|2023-04-14 10:54:07|t3_12lt8za|t3_12lt8za|
|2023-12-05 16:39:49|t3_18bg4m8|t3_18bg4m8|
|2023-08-14 14:09:00|t3_15qvrmj|

In [15]:
# creating new alias, so I know what to drop
joined_null = joined_null.select(col('estimated_time').alias('estimated_time'), col('id').alias('id_b'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
joined_null.printSchema()
print('Total Rows: %d' % joined_null.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- estimated_time: timestamp (nullable = true)
 |-- id_b: string (nullable = true)

Total Rows: 5743

In [17]:
# remembering how many rows data had originally
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Rows: 146173

In [18]:
# joining on left outer
data = data.join(joined_null, 
                          data['id'] == joined_null['id_b'], 
                          how="leftOuter")\

# using coalesce to combine times by getting first non-null value
data = data.withColumn('time', coalesce(data['time'], data['estimated_time']))

# making sure it's still the correct amount of rows
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Rows: 146173

In [19]:
# dropping cols I no longer need
data = data.drop('id_b', 'estimated_time').persist()
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- body: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- time: timestamp (nullable = true)

In [20]:
data.where(data['id'] == 'k8dhz8e').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+-----+-------+----------+------------+-----------------+-------------------+
|                body|          author|score|     id|   link_id|   subreddit|author_flair_text|               time|
+--------------------+----------------+-----+-------+----------+------------+-----------------+-------------------+
|Leftists have bee...|Sauvignon_Bleach|  -60|k8dhz8e|t3_17qp9w5|Conservative|     Conservative|2023-11-08 16:22:07|
+--------------------+----------------+-----+-------+----------+------------+-----------------+-------------------+

This is one of the comments from the original null time chart, and as you can see it now has the time, and is not duplicated. Now I can see the overall range of comments.

In [21]:
null_t_count = data.where(data['time'] == 'null').count()
print('Null Time Count:', null_t_count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Null Time Count: 0

In [22]:
year = data.withColumn('year', year('time'))

posts_per_year = year.groupBy('year').agg(count('*').alias('count'))

posts_per_year.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|year|count|
+----+-----+
|2018| 8430|
|2010|   77|
|2011|  214|
|2017| 5383|
|2013| 2835|
|2020|15153|
|2015| 1909|
|2023| 9661|
|2014| 1023|
|2012| 2169|
|2016| 4430|
|2021|29346|
|2022|52151|
|2019|13374|
|2008|    5|
|2009|   13|
+----+-----+

# User Flares
An 'author_flair_text' is a title that goes along with your Reddit username along every post that you make within the sub. They are pretty good indicators of who is active in the sub, and in this case, good indicators of who is definitley a conservative, and not someone who just posts on the subreddit. This will become more obvious when I show the example.

In [23]:
flares = data.where(data['author_flair_text'] != 'null')
print('Flares with the Subreddit:', flares.count())

print()
print('An example of some flares:')
flares.select('author_flair_text').distinct().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Flares with the Subreddit: 66932

An example of some flares:
+----------------------------+
|author_flair_text           |
+----------------------------+
|Conservative CPA            |
|Libertarian Conservative    |
|MAGA                        |
|Conservatus Maximus         |
|ΜΟΛΩΝ ΛΑΒΕ                  |
|Liberty-Minded              |
|Constitution                |
|We the People               |
|God, Family, Country        |
|Libertarian-Conservative    |
|Strict Constitutionalist    |
|Live Free or Die            |
|Mei the Wall be with you    |
|Anti-Censorship &amp; Pro 2A|
|Annex Judea &amp; Samaria   |
|Traditionalist Conservative |
|Christian                   |
|paleocon                    |
|Limbaugh Conservative       |
|2A!                         |
+----------------------------+
only showing top 20 rows

In [24]:
high_scores = data.orderBy('score', ascending=False)
high_scores.select('body','score', 'time').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------------+
|body                                                                                                                                                                                                         

In [25]:
low_scores = data.orderBy('score', ascending=True)
low_scores.select('body','score', 'time').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------------+
|body                                                                                                                                                                                                                         

In [26]:
# lastly, want to get rid of any comments that won't hold enough meaning within the dataset, so dropping smaller responses
data = data.filter(size(split(col('body'), ' ')) >= 2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
print('Total Rows: %d' % data.count())
print('Total Columns: %d' % len(data.dtypes))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Rows: 142924
Total Columns: 8

In [29]:
# writing to bucket
bucket = 's3://131313113finalproject/cleaned_data_2'

data.write.parquet(bucket, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…